In [139]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import glob

In [2]:
import dash
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px

In [188]:
from PIL import Image, ImageDraw, ImageFilter

In [185]:
def get_analysis(watchlist):
    
    def get_decade(year):
        return year - year%10

    def split_genres(genres):
        return genres.split(",")
    
    def add_img_path(director):
        director = str(director).replace(" ","_")
        director = "assets/img/"+director+".jpg"
        return director
    
    watchlist["Year"] = watchlist["Year"].apply(get_decade)
    watchlist["Genres"] = watchlist["Genres"].apply(split_genres)
    watchlist["img_path"] = watchlist["Directors"].apply(add_img_path)
    
    genres_dict = dict()
    for i in range(len(watchlist["Genres"])):
            watchlist['Title'].loc[i] = watchlist['Title'].loc[i].replace("'","")
            for k in watchlist["Genres"].loc[i]:
                if k[0] == " ":
                    k = k.replace(" ","")
                if k not in genres_dict.keys():
                    genres_dict[k] = {'count':0,'Title':[]}
                genres_dict[k]['count'] = genres_dict[k]['count']  +1
                genres_dict[k]['Title'].append(watchlist["Title"].iloc[i])
                
    genre_count_dict = dict()
    for k in genres_dict.keys():
        genre_count_dict[k] = genres_dict[k]['count']
    genre_count_dict = {k: genre_count_dict[k] for k in sorted(genre_count_dict, key=genre_count_dict.get, reverse=True)}
                
    decade_dict = watchlist.groupby(['Year']).agg({'Title':"count"}).to_dict()['Title']
    top_directors = watchlist.groupby(['Directors']).agg({'Title':"count"}).to_dict()['Title']
    top_directors = {k: top_drectors[k] for k in sorted(top_drectors, key=top_drectors.get, reverse=True)}
    top_directors = list(top_directors.items())
    
    avg_rating_imdb = "%.2f" %np.mean(watchlist['IMDb Rating'])
    avg_my_rating = "%.2f" %np.mean(watchlist[watchlist['Your Rating'].notnull()]['Your Rating'])
    sum_run_time = sum(watchlist[watchlist['Runtime (mins)'].notnull()]['Runtime (mins)'])
    total_movies = len(watchlist)
    
    
    general_info = [avg_rating_imdb, avg_my_rating, sum_run_time, total_movies]
    
    return (general_info, decade_dict, genres_dict, genre_count_dict, watchlist)


In [186]:
watchlist = pd.read_csv("/Users/choobani/Downloads/Masters/4-Papers/watchlist/datasets/WATCHLIST_vahab.csv",encoding="ISO-8859-1")
general_info, decade_dict, genres_dict, genre_count_dict, watchlist =  get_analysis(watchlist)

/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [182]:
watchlist

,Position,Const,Created,Modified,Description,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,Num Votes,Release Date,Directors,Your Rating,Date Rated,img_path
0,1,tt0108052,2013-12-02,2013-12-02,NaN,Schindlers List,https://www.imdb.com/title/tt0108052/,movie,8.9,195,1990,"[Biography, Drama, History]",1166225.0,1993-11-30,Steven Spielberg,NaN,NaN,assets/img/Steven_Spielberg.jpg
1,2,tt0361748,2014-05-30,2014-05-30,NaN,Inglourious Basterds,https://www.imdb.com/title/tt0361748/,movie,8.3,153,2000,"[Adventure, Drama, War]",1210881.0,2009-05-20,Quentin Tarantino,NaN,NaN,assets/img/Quentin_Tarantino.jpg
2,3,tt0360717,2014-05-30,2014-05-30,NaN,King Kong,https://www.imdb.com/title/tt0360717/,movie,7.2,187,2000,"[Action, Adventure, Drama, Romance]",381133.0,2005-12-05,Peter Jackson,NaN,NaN,assets/img/Peter_Jackson.jpg
3,4,tt0110912,2014-07-28,2014-07-28,NaN,Pulp Fiction,https://www.imdb.com/title/tt0110912/,movie,8.9,154,1990,"[Crime, Drama]",1757293.0,1994-05-21,Quentin Tarantino,NaN,NaN,assets/img/Quentin_Tarantino.jpg
4,5,tt0407887,2014-07-28,2014-07-28,NaN,The Departed,https://www.imdb.com/title/tt0407887/,movie,8.5,151,2000,"[Crime, Drama, Thriller]",1143762.0,2006-09-26,Martin Scorsese,NaN,NaN,assets/img/Martin_Scorsese.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,756,tt0134273,2020-05-04,2020-05-04,NaN,8MM,https://www.imdb.com/title/tt0134273/,movie,6.5,123,1990,"[Mystery, Thriller]",120216.0,1999-02-19,Joel Schumacher,NaN,NaN,assets/img/Joel_Schumacher.jpg
756,757,tt1727824,2020-05-06,2020-05-06,NaN,Bohemian Rhapsody,https://www.imdb.com/title/tt1727824/,movie,8.0,134,2010,"[Biography, Drama, Music]",424127.0,2018-10-23,Bryan Singer,NaN,NaN,assets/img/Bryan_Singer.jpg
757,758,tt0486655,2020-05-07,2020-05-07,NaN,Stardust,https://www.imdb.com/title/tt0486655/,movie,7.6,127,2000,"[Adventure, Family, Fantasy, Romance]",247398.0,2007-07-29,Matthew Vaughn,NaN,NaN,assets/img/Matthew_Vaughn.jpg
758,759,tt0405422,2020-05-16,2020-05-16,NaN,The 40 Year Old Virgin,https://www.imdb.com/title/tt0405422/,movie,7.1,116,2000,"[Comedy, Romance]",381792.0,2005-08-11,Judd Apatow,NaN,NaN,assets/img/Judd_Apatow.jpg


In [150]:
top_drectors = watchlist.groupby(['Directors']).agg({'Title':"count"}).to_dict()['Title']
top_drectors = {k: top_drectors[k] for k in sorted(top_drectors, key=top_drectors.get, reverse=True)}

list(top_drectors.items())[0][0]
    

'Steven Spielberg'

In [183]:
list(watchlist.loc[watchlist['Directors']=='Steven Spielberg']['img_path'])[0]

'assets/img/Steven_Spielberg.jpg'

## Compress Images

In [256]:
path = r'/Users/choobani/Downloads/Masters/4-Papers/watchlist/img_raw/' # use your path
all_files = glob.glob(path + "/*.jpg")

for img in all_files:
    
    file_name = img.split('/')[8][:-4]
    
    img = Image.open(img)

    width, height = img.size;

    new_width  = 300
    new_height = round(new_width * height / width)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)

    img.save('/Users/choobani/Downloads/Masters/4-Papers/watchlist/static/img/dir_img/{}.jpg'.format(file_name), quality=95)


## Create Square Images

In [261]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))


In [265]:
path = r'/Users/choobani/Downloads/Masters/4-Papers/watchlist/img_raw/' # use your path
all_files = glob.glob(path + "/*.jpg")

for img in all_files:
    
    file_name = img.split('/')[8][:-4]
    
    img = Image.open(img)

    width, height = img.size;

    new_width  = 300
    new_height = round(new_width * height / width)
    img = img.resize((new_width, new_height), Image.ANTIALIAS)

    thumb_width = 300
    im_thumb = crop_center(img, thumb_width, thumb_width)

    im_thumb.save('/Users/choobani/Downloads/Masters/4-Papers/watchlist/static/img/dir_img/{}.jpg'.format(file_name))


In [117]:
watchlist = pd.read_csv("/Users/choobani/Downloads/WATCHLIST.csv",encoding="ISO-8859-1")


def get_decade(year):
    return year - year%10

def split_genres(genres):
    return genres.split(",")

watchlist["Year"] = watchlist["Year"].apply(get_decade)
watchlist["Genres"] = watchlist["Genres"].apply(split_genres)

genres_dict = dict()
for i in range(len(watchlist["Genres"])):
        watchlist['Title'].loc[i] = watchlist['Title'].loc[i].replace("'","")
        for k in watchlist["Genres"].loc[i]:
            if k[0] == " ":
                k = k.replace(" ","")
            if k not in genres_dict.keys():
                genres_dict[k] = {'count':0,'movies':{watchlist["Title"].iloc[i]:0}}
            genres_dict[k]['count'] = genres_dict[k]['count']  +1
            
            genres_dict[k]['movies'][watchlist["Title"].iloc[i]] = watchlist["Year"].iloc[i]


In [118]:
genre_count_dict = dict()
for k in genres_dict.keys():
    genre_count_dict[k] = genres_dict[k]['count']
genre_count_dict = {k: genre_count_dict[k] for k in sorted(genre_count_dict, key=genre_count_dict.get, reverse=True)}


In [119]:
def get_titles(title_list):

    title_data = []
    for i in range(len(title_list)):
        rank = float((watchlist.loc[watchlist['Title']== title_list[i]]['IMDb Rating']).to_list()[0])
        temp_dict = {'Title':title_list[i], 'Rank': rank}
        title_data.append(temp_dict)
    #return title_data
    return sorted(title_data, key=itemgetter('Rank'), reverse=True)


{'Drama': 196,
 'Adventure': 90,
 'Thriller': 89,
 'Action': 69,
 'Fantasy': 61,
 'Sci-Fi': 58,
 'Mystery': 56,
 'Comedy': 53,
 'Crime': 52,
 'Biography': 40,
 'Romance': 38,
 'Family': 38,
 'Animation': 29,
 'War': 20,
 'History': 17,
 'Music': 10,
 'Documentary': 6,
 'Horror': 5,
 'Musical': 4,
 'Sport': 4,
 'Western': 2,
 'News': 1,
 'Short': 1}

In [121]:
def get_titles(genres_dict, genre, decade):

    title_list = []
    for k1 in genres_dict.keys():
        if k1 == genre :
            for k2 in genres_dict[k1]['movies']:
                if genres_dict[k1]['movies'][k2] == decade:
                    title_list.append(k2)
    
    title_data = []
    for i in range(len(title_list)):
        rank = float((watchlist.loc[watchlist['Title']== title_list[i]]['IMDb Rating']).to_list()[0])
        temp_dict = {'Title':title_list[i], 'Rank': rank}
        title_data.append(temp_dict)
    #return title_data
    return sorted(title_data, key=itemgetter('Rank'), reverse=True)


In [120]:
genres_dict(genres_dict, 'Drama', 1990)

KeyError: 'Title'

In [127]:
genres_dict

{'Drama': {'count': 196,
  'movies': {'The Shawshank Redemption': 1990,
   'The Godfather': 1970,
   'The Godfather: Part II': 1970,
   'The Dark Knight': 2000,
   '12 Angry Men': 1950,
   'Schindlers List': 1990,
   'The Lord of the Rings: The Return of the King': 2000,
   'Pulp Fiction': 1990,
   'Fight Club': 1990,
   'The Lord of the Rings: The Fellowship of the Ring': 2000,
   'Forrest Gump': 1990,
   'The Lord of the Rings: The Two Towers': 2000,
   'One Flew Over the Cuckoos Nest': 1970,
   'Goodfellas': 1990,
   'Shichinin no samurai': 1950,
   'Se7en': 1990,
   'Cidade de Deus': 2000,
   'La vita è bella': 1990,
   'Saving Private Ryan': 1990,
   'Léon': 1990,
   'Interstellar': 2010,
   'American History X': 1990,
   'The Pianist': 2000,
   'Intouchables': 2010,
   'The Departed': 2000,
   'Whiplash': 2010,
   'Gladiator': 2000,
   'Nuovo Cinema Paradiso': 1980,
   'American Beauty': 1990,
   'Das Boot': 1980,
   'Braveheart': 1990,
   'Requiem for a Dream': 2000,
   'A Clock

In [123]:
genre = 'Drama'
decade = 1990

title_data = []
for k1 in genres_dict.keys():
    for k2 in genres_dict[k1]['movies']:
        if genres_dict[k1]['movies'][k2] == decade:
            title_data.append(k2)

In [124]:
title_data

['The Shawshank Redemption',
 'Schindlers List',
 'Pulp Fiction',
 'Fight Club',
 'Forrest Gump',
 'Goodfellas',
 'Se7en',
 'La vita è bella',
 'Saving Private Ryan',
 'Léon',
 'American History X',
 'American Beauty',
 'Braveheart',
 'Casino',
 'The Sixth Sense',
 'The Truman Show',
 'Before Sunrise',
 'The Green Mile',
 'Titanic',
 'Good Will Hunting',
 'Twin Peaks',
 'The Godfather: Part III',
 'Scent of a Woman',
 'The Devils Advocate',
 'Eyes Wide Shut',
 'Heat',
 'Cube',
 'Patch Adams',
 'Bicentennial Man',
 'Trainspotting',
 'La double vie de Véronique',
 'The Remains of the Day']

In [62]:
genre_options = []

for k in genre_count_dict.keys():
    temp_dict = {'label':k,'value':k}
    genre_options.append(temp_dict)
    
genre_options

[{'label': 'Drama', 'value': 'Drama'},
 {'label': 'Thriller', 'value': 'Thriller'},
 {'label': 'Action', 'value': 'Action'},
 {'label': 'Crime', 'value': 'Crime'},
 {'label': 'Comedy', 'value': 'Comedy'},
 {'label': 'Adventure', 'value': 'Adventure'},
 {'label': 'Sci-Fi', 'value': 'Sci-Fi'},
 {'label': 'Mystery', 'value': 'Mystery'},
 {'label': 'Romance', 'value': 'Romance'},
 {'label': 'Fantasy', 'value': 'Fantasy'},
 {'label': 'Biography', 'value': 'Biography'},
 {'label': 'Family', 'value': 'Family'},
 {'label': 'Animation', 'value': 'Animation'},
 {'label': 'War', 'value': 'War'},
 {'label': 'History', 'value': 'History'},
 {'label': 'Horror', 'value': 'Horror'},
 {'label': 'Sport', 'value': 'Sport'},
 {'label': 'Musical', 'value': 'Musical'},
 {'label': 'Music', 'value': 'Music'},
 {'label': 'Western', 'value': 'Western'},
 {'label': 'Film-Noir', 'value': 'Film-Noir'},
 {'label': 'Documentary', 'value': 'Documentary'},
 {'label': 'Short', 'value': 'Short'},
 {'label': 'News', 'val

In [63]:
genres_dict

{'Biography': {'count': 86,
  'Title': ["Schindler's List",
   'Braveheart',
   'Lawrence of Arabia',
   'Gandhi',
   'In the Name of the Father',
   'Hotel Rwanda',
   'Rush',
   'Goodfellas',
   'Papillon',
   'Intouchables',
   'The Social Network',
   'Dallas Buyers Club',
   'The Fighter',
   'Patch Adams',
   'Moneyball',
   'The Blind Side',
   'The Queen',
   'Mar adentro',
   'My Left Foot: The Story of Christy Brown',
   'Lincoln',
   'The Wolf of Wall Street',
   'American Gangster',
   'The Straight Story',
   'Man on the Moon',
   'Butch Cassidy and the Sundance Kid',
   'Raging Bull',
   'Catch Me If You Can',
   'The Imitation Game',
   'The Theory of Everything',
   'Selma',
   'Ray',
   'The Aviator',
   'Serpico',
   'J. Edgar',
   'Amadeus',
   'Der Untergang',
   'The Elephant Man',
   'Into the Wild',
   'Yip Man',
   'Wild',
   'Unbroken',
   'The Big Short',
   'Spotlight',
   'Steve Jobs',
   'Trumbo',
   'Joy',
   'Everest',
   'The Walk',
   'Black Mass',
   '

In [64]:
general_info

['7.64', 'nan', 89777, 760]

In [15]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='genres_pie',
        figure= px.pie(values=list(genres_dict.values()), names=list(genres_dict.keys()))
    )
], className='ten columns offset-by-one')



if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 452-073-840
Debugger PIN: 452-073-840
Debugger PIN: 452-073-840
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [73]:
import dash
import dash_table
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict('records'),
)

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 718-376-904
Debugger PIN: 718-376-904
Debugger PIN: 718-376-904
Debugger PIN: 718-376-904
Debugger PIN: 718-376-904
Debugger PIN: 718-376-904
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


In [8]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')
columns=[{"name": i, "id": i} for i in df.columns],
data=df.to_dict('records')

In [10]:
columns

([{'name': 'State', 'id': 'State'},
  {'name': 'Number of Solar Plants', 'id': 'Number of Solar Plants'},
  {'name': 'Installed Capacity (MW)', 'id': 'Installed Capacity (MW)'},
  {'name': 'Average MW Per Plant', 'id': 'Average MW Per Plant'},
  {'name': 'Generation (GWh)', 'id': 'Generation (GWh)'}],)

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather: Part II',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'Fight Club',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Forrest Gump',
 'The Lord of the Rings: The Two Towers',
 "One Flew Over the Cuckoo's Nest",
 'Goodfellas',
 'Shichinin no samurai',
 'Se7en',
 'Cidade de Deus',
 'La vita è bella',
 'Saving Private Ryan',
 'Léon',
 'Interstellar',
 'American History X',
 'The Pianist',
 'Intouchables',
 'The Departed',
 'Whiplash',
 'Gladiator',
 'Nuovo Cinema Paradiso',
 'American Beauty',
 'Das Boot',
 'Braveheart',
 'Requiem for a Dream',
 'A Clockwork Orange',
 'Taxi Driver',
 'Scarface',
 'Incendies',
 'A Beautiful Mind',
 'Casino',
 'Inside Out',
 'The Elephant Man',
 'The Wolf of Wall Street',
 'V for Vendetta',
 'The Sixth Sense',
 'Mary and Max',
 'Hacksaw Ridge',
 'The Truman Show',
 'Before Sunrise',
 'Spotlight',
 'Catch Me 

Series([], Name: IMDb Rating, dtype: float64)

In [55]:
from operator import itemgetter
def get_titles(title_list):

    title_data = []
    for i in range(len(title_list)):
        temp_dict = {'Title':title_list[i], 'Rank': float((watchlist.loc[watchlist['Title']== title_list[i]]['IMDb Rating']).to_list()[0])}
        title_data.append(temp_dict)
    #return title_data
    return sorted(title_data, key=itemgetter('Rank'), reverse=True)    


get_titles(genres_dict['Drama']['Title']) 



[{'Title': 'Breaking Bad', 'Rank': 9.5},
 {'Title': 'Chernobyl', 'Rank': 9.4},
 {'Title': 'The Shawshank Redemption', 'Rank': 9.3},
 {'Title': 'The Godfather', 'Rank': 9.2},
 {'Title': 'The Godfather: Part II', 'Rank': 9.0},
 {'Title': 'The Dark Knight', 'Rank': 9.0},
 {'Title': 'True Detective', 'Rank': 9.0},
 {'Title': '12 Angry Men', 'Rank': 8.9},
 {'Title': "Schindler's List", 'Rank': 8.9},
 {'Title': 'The Lord of the Rings: The Return of the King', 'Rank': 8.9},
 {'Title': 'Pulp Fiction', 'Rank': 8.9},
 {'Title': 'Fight Club', 'Rank': 8.8},
 {'Title': 'The Lord of the Rings: The Fellowship of the Ring', 'Rank': 8.8},
 {'Title': 'Forrest Gump', 'Rank': 8.8},
 {'Title': 'Twin Peaks', 'Rank': 8.8},
 {'Title': 'Peaky Blinders', 'Rank': 8.8},
 {'Title': 'Black Mirror', 'Rank': 8.8},
 {'Title': 'The Lord of the Rings: The Two Towers', 'Rank': 8.7},
 {'Title': "One Flew Over the Cuckoo's Nest", 'Rank': 8.7},
 {'Title': 'Goodfellas', 'Rank': 8.7},
 {'Title': 'Westworld', 'Rank': 8.7},
 {'

In [31]:
sorted(l, key=itemgetter('name'), reverse=True)

0    9.2
Name: IMDb Rating, dtype: float64

In [15]:
genres_dict['Drama']['Title']

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather: Part II',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'Fight Club',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Forrest Gump',
 'The Lord of the Rings: The Two Towers',
 "One Flew Over the Cuckoo's Nest",
 'Goodfellas',
 'Shichinin no samurai',
 'Se7en',
 'Cidade de Deus',
 'La vita è bella',
 'Saving Private Ryan',
 'Léon',
 'Interstellar',
 'American History X',
 'The Pianist',
 'Intouchables',
 'The Departed',
 'Whiplash',
 'Gladiator',
 'Nuovo Cinema Paradiso',
 'American Beauty',
 'Das Boot',
 'Braveheart',
 'Requiem for a Dream',
 'A Clockwork Orange',
 'Taxi Driver',
 'Scarface',
 'Incendies',
 'A Beautiful Mind',
 'Casino',
 'Inside Out',
 'The Elephant Man',
 'The Wolf of Wall Street',
 'V for Vendetta',
 'The Sixth Sense',
 'Mary and Max',
 'Hacksaw Ridge',
 'The Truman Show',
 'Before Sunrise',
 'Spotlight',
 'Catch Me 

In [ ]:
sudo lsof -i:8050

In [222]:
top_drectors

{'Steven Spielberg': 13,
 'Alfred Hitchcock': 10,
 'Martin Scorsese': 10,
 'Akira Kurosawa': 9,
 'Hayao Miyazaki': 8,
 'Stanley Kubrick': 8,
 'Billy Wilder': 7,
 'Clint Eastwood': 7,
 'Alejandro G. Iñárritu': 6,
 'Ethan Coen, Joel Coen': 6,
 'Quentin Tarantino': 6,
 'Ridley Scott': 6,
 'Tim Burton': 6,
 'Bryan Singer': 5,
 'Guillermo del Toro': 5,
 'James Mangold': 5,
 'Robert Zemeckis': 5,
 'Bong Joon Ho': 4,
 'Danny Boyle': 4,
 'David Fincher': 4,
 'Francis Lawrence': 4,
 'George Lucas': 4,
 'Gore Verbinski': 4,
 'Guy Ritchie': 4,
 'Ingmar Bergman': 4,
 'James Cameron': 4,
 'Jason Reitman': 4,
 'Justin Lin': 4,
 'Richard Linklater': 4,
 'Sergio Leone': 4,
 'Todd Phillips': 4,
 'Adam McKay': 3,
 'Andrei Tarkovsky': 3,
 'Antoine Fuqua': 3,
 'Chan-wook Park': 3,
 'Charles Chaplin': 3,
 'David Ayer': 3,
 'David O. Russell': 3,
 'Duncan Jones': 3,
 'Edgar Wright': 3,
 'Federico Fellini': 3,
 'Frank Capra': 3,
 'George Clooney': 3,
 'Howard Hawks': 3,
 'Jim Sheridan': 3,
 'John Carney': 3,